In [ ]:
!pip install --no-cache-dir transformers
import sys
import csv
from transformers import pipeline
from nltk.corpus import wordnet
import nltk
nltk.download('wordnet')

class BSNode:

    def __init__(self, data, prob):
        self.data = data
        self.prob = prob
        self.exclusion = []
        self.explored = False
        self.left = None
        self.right = None


    def isLeaf(self):
        if(self.left):
            return False
        if(self.right):
            return False
        return True


    #prints out node, left, data/prob/explored, then right
    def printNode(self):
        if(self.isLeaf()):
            print(self.data, self.prob, self.explored)
        else:
            if(self.left):
                self.left.printNode()

            print(self.data, self.prob, self.explored)

            if(self.right):
                self.right.printNode()



    #returns the node with the lowest probability
    # which hasn't yet been explored, if every node has been explored
    # returns None
    def lowestProb(self):
        if(self.isLeaf()): # cannot explore further
            if(self.explored): # already explored, exit
                return None
            else:
                return self # only one node here, so must be the lowest

        else: #explore right and left
            left_low = right_low = None

            if(self.left):
                left_low = self.left.lowestProb()
            if(self.right):
                right_low = self.right.lowestProb()

            if(not left_low and not right_low): #both branches explored, which means this node already is fully explored
                self.explored = True
                return None

            if(not left_low):
                return right_low
            if(not right_low):
                return left_low

            # finally if here, both left and right have not been explored
            # so chooose the lower probability
            if(left_low.prob < right_low.prob):
                return left_low
            else:
                return right_low






def binary_select_iterative_BSNode(classifier, query, attack_class = 0, BSStruct = None, initial_prob = None): #for analysis

    query_count = 0

    # set up initial root BSNode
    if(not BSStruct):
        if(not initial_prob):
            # get initial probability for query
            initial_prob = classifier(query)[0][attack_class]["score"]

        query_count += 1

        BSStruct = BSNode(query, initial_prob)



    initial_prob = BSStruct.prob

    # search to find lowest prob node, which has also not been explored
    cur_struct = BSStruct.lowestProb()
    final_prob = initial_prob

    split_query = query.split()


    if(len(cur_struct.exclusion) == 1): # only 1 word, no need to explore more, just return
        cur_struct.explored = True # exploring!
        return cur_struct.exclusion[0], cur_struct.prob, 0, BSStruct #exclusion list with only 1 item is that word's position in the text
    else: # set up start and end
        if(len(cur_struct.exclusion) == 0): # root node, no exclusion, entire text
            start = 0
            end = len(split_query) - 1#makes slicing easier
        else:
            start = cur_struct.exclusion[0]
            end = start + len(cur_struct.exclusion) - 1


    while start < end:
        mid = start + (end - start) // 2

        first_exclusion = list(range(start, mid+1))
        second_exclusion = list(range(mid+1, end+1))

        first_list = []
        second_list =[]

        for i in range(len(split_query)):
          if(i not in first_exclusion):
            first_list.append(split_query[i])
        first_part = ' '.join(first_list)

        for i in range(len(split_query)):
          if(i not in second_exclusion):
            second_list.append(split_query[i])
        second_part = ' '.join(second_list)

        first_prob = classifier(first_part)[0][attack_class]["score"]
        second_prob = classifier(second_part)[0][attack_class]["score"]
        query_count += 2  # Increment for both sentiment analysis requests

        # add information to Binary structure as we get it
        cur_struct.left = BSNode(' '.join([split_query[i] for i in first_exclusion]), first_prob)
        cur_struct.right = BSNode(' '.join([split_query[i] for i in second_exclusion]), second_prob)
        cur_struct.left.exclusion = first_exclusion
        cur_struct.right.exclusion = second_exclusion



        first_drop = initial_prob - first_prob
        second_drop = initial_prob - second_prob

        if first_drop > second_drop:
            if(len(first_exclusion) == 1):
                # need to note that we are exploring this node so we don't again!!!!
                cur_struct.left.explored = True

            final_prob = first_prob

            cur_struct = cur_struct.left
            start = first_exclusion[0]
            end = first_exclusion[-1]
        else:
            if(len(second_exclusion) == 1):
                cur_struct.right.explored = True

            final_prob = second_prob

            cur_struct = cur_struct.right
            start = second_exclusion[0]
            end = second_exclusion[-1]


        most_influential_pos = start


    return most_influential_pos, final_prob, query_count, BSStruct



def get_synonyms(word):
    synonyms = set()

    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name())

    return list(synonyms - {word})

# takes in an output from textclassifier and returns the highest probability
# needed for multiclass problems
def pred_class(preds):
    pred = 0
    score = preds[0]['score']

    for i in range(1, len(preds)):
        cur_score = preds[i]['score']
        if(cur_score > score):
            pred = i
            score = cur_score

    return pred

def wordnet_replace(classifier, query, initial_prob, replace_pos, attack_class):
    split_query = query.split()
    replace_word = split_query[replace_pos]
    syns = get_synonyms(replace_word)
    query_count = 0

    # if no synonyms found, can't change this word!
    if(len(syns) == 0):
        return False, query, initial_prob, query_count

    #print(syns)
    prob_list = []
    #test each synonym, choose first to cause failure, or else choose synonym which caused the greater probability drop
    for cur_syn in syns:
        cur_query = split_query[:replace_pos] + [cur_syn] + split_query[replace_pos +1:]
        cur_query = ' '.join(cur_query)

        cur_preds = classifier(cur_query)[0]
        #print(cur_query, cur_preds)
        query_count += 1
        cur_prob = cur_preds[attack_class]['score']
        cur_label = pred_class(cur_preds)

        # this means the attack flipped the label
        if(cur_label != attack_class):
            return True, cur_query, cur_prob, query_count
        else:# add to prob list for later
            prob_list.append((cur_prob, cur_query))


    # here we didn't find any synonyms to cause failure, so we choose the one which caused the greater drop
    great_drop = 0
    choice = 0
    for i in range(len(prob_list)):
        prob_drop = initial_prob - prob_list[i][0]
        if(prob_drop > great_drop):
            choice = i
            great_drop = prob_drop

    return False, prob_list[choice][1], prob_list[choice][0], query_count


# takes in a text and attempts to flip the label with binary selection and wordnet synonym replacement
def BS_WNR(classifier, query, attack_class):
    done = False
    query_count = 0
    cur_struct = None
    initial_prob = classifier(query)[0][attack_class]["score"]
    query_count += 1

    cur_query = query

    while(not done):
        # call binary select
        #
        if(cur_struct):
            replace_pos, final_prob, queries, cur_struct = binary_select_iterative_BSNode(classifier, query, attack_class, BSStruct = cur_struct, initial_prob = initial_prob)
        else:
            replace_pos, final_prob, queries, cur_struct = binary_select_iterative_BSNode(classifier, query, attack_class, initial_prob = initial_prob)




        #cur_struct.printNode()

        #add new amount of queries
        query_count += queries


        # try and replace with wordnet replace
        success, cur_query, cur_prob, queries = wordnet_replace(classifier, cur_query, initial_prob, replace_pos, attack_class)


        query_count += queries
        #print(success, cur_query, cur_prob, query_count)

        if(success):
            return True, cur_query, query_count, cur_prob
        else:
            #check if everything explored
            if(not cur_struct.lowestProb()):
                done = True

    return False, cur_query, query_count, cur_prob


def GreedySelect(classifier, query, attack_class = 0, initial_prob = None):

    if(not initial_prob):
        initial_prob = classifier(query)[0][attack_class]["score"]

    prob_drops = {}

    split_query = query.split()
    # remove each word and then note the drop in probability
    for i in range(len(split_query)):
        cur_query = ' '.join(split_query[:i] + split_query[i+1:])
        cur_prob = classifier(cur_query)[0][attack_class]["score"]
        prob_drops[i] = initial_prob - cur_prob


    return prob_drops




def GS_WNR(classifier, query, attack_class):
    done = False
    query_count = 0
    cur_struct = None
    initial_prob = classifier(query)[0][attack_class]["score"]

    query_count += 1

    cur_query = query

    prob_drops = GreedySelect(classifier, query, attack_class, initial_prob)
    query_count += len(prob_drops)

    prob_sorted = dict(sorted(prob_drops.items(), key=lambda item: item[1], reverse=True))

    while(not done):

        replace_pos = list(prob_sorted.keys())[0]
        prob_sorted.pop(replace_pos)

        # try and replace with wordnet replace
        success, cur_query, cur_prob, queries = wordnet_replace(classifier, cur_query, initial_prob, replace_pos, attack_class)

        query_count += queries
        #print(success, cur_query, cur_prob, query_count)

        if(success):
            return True, cur_query, query_count, cur_prob
        else:
            #check if everything explored
            if(len(prob_sorted) == 0):
                done = True

    return False, cur_query, query_count, cur_prob




def testimdb():
    classifier =  pipeline("text-classification", model="textattack/distilbert-base-uncased-imdb", max_length=512, truncation = True, return_all_scores = True)
    imdbin = csv.reader(open("/content/drive/MyDrive/Colab Notebooks/imdb_test_1000.tsv"), delimiter = '\t')
    binaryout = csv.writer(open("/content/drive/MyDrive/Colab Notebooks/imdb_test_BS-WNR.tsv", 'w'), delimiter = '\t')
    greedyout = csv.writer(open("/content/drive/MyDrive/Colab Notebooks/imdb_test_GS-WNR.tsv", 'w'), delimiter = '\t')

    cur_id = 0
    for cur in imdbin:
        #make sure the classifier is not already failing
        cur_pred = classifier(cur)[0]
        predicted = pred_class(cur_pred)
        if(predicted != int(cur[1])):
            success = 'Skipped'
            binaryout.writerow([cur_id, success, cur[0], 0, 0.0])
            greedyout.writerow([cur_id, success, cur[0], 0, 0.0])
            cur_id += 1
            continue

        success, final_query, query_count, final_prob = BS_WNR(classifier, cur[0], int(cur[1]))
        binaryout.writerow([cur_id, success, final_query, query_count, final_prob])

        success, final_query, query_count, final_prob = GS_WNR(classifier, cur[0], int(cur[1]))
        greedyout.writerow([cur_id, success, final_query, query_count, final_prob])

        cur_id += 1
        if(cur_id > 9):
            break


#testimdb()

#classifier =  pipeline("text-classification", model="textattack/distilbert-base-uncased-imdb", max_length=512, truncation = True, return_all_scores = True)

#query_count = 0



#text = 'the worst movie ever'
#success, final_query, query_count, final_prob = BS_WNR(classifier, text, 0)
#print(success, final_query, query_count, final_prob)

#text = 'the worst movie ever . . . not good once'
#success, final_query, query_count, final_prob = BS_WNR(classifier, text, 0)
#print(success, final_query, query_count, final_prob)

#text = 'the worst movie ever . . . not good once'
#success, final_query, query_count, final_prob = GS_WNR(classifier, text, 0)
#print(success, final_query, query_count, final_prob)

#text = '''"I Am Curious: Yellow" is a risible and pretentious steaming pile. It doesn't matter what one's political views are because this film can hardly be taken seriously on any level. As for the claim that frontal male nudity is an automatic NC-17, that isn't true. I've seen R-rated films with male nudity. Granted, they only offer some fleeting views, but where are the R-rated films with gaping vulvas and flapping labia? Nowhere, because they don't exist. The same goes for those crappy cable shows: schlongs swinging in the breeze but not a clitoris in sight. And those pretentious indie movies like The Brown Bunny, in which we're treated to the site of Vincent Gallo's throbbing johnson, but not a trace of pink visible on Chloe Sevigny. Before crying (or implying) "double-standard" in matters of nudity, the mentally obtuse should take into account one unavoidably obvious anatomical difference between men and women: there are no genitals on display when actresses appears nude, and the same cannot be said for a man. In fact, you generally won't see female genitals in an American film in anything short of porn or explicit erotica. This alleged double-standard is less a double standard than an admittedly depressing ability to come to terms culturally with the insides of women's bodies.'''
#success, final_query, query_count, final_prob = BS_WNR(classifier, text, 0)
#print(success, final_query, query_count, final_prob)

#text = '''"I Am Curious: Yellow" is a risible and pretentious steaming pile. It doesn't matter what one's political views are because this film can hardly be taken seriously on any level. As for the claim that frontal male nudity is an automatic NC-17, that isn't true. I've seen R-rated films with male nudity. Granted, they only offer some fleeting views, but where are the R-rated films with gaping vulvas and flapping labia? Nowhere, because they don't exist. The same goes for those crappy cable shows: schlongs swinging in the breeze but not a clitoris in sight. And those pretentious indie movies like The Brown Bunny, in which we're treated to the site of Vincent Gallo's throbbing johnson, but not a trace of pink visible on Chloe Sevigny. Before crying (or implying) "double-standard" in matters of nudity, the mentally obtuse should take into account one unavoidably obvious anatomical difference between men and women: there are no genitals on display when actresses appears nude, and the same cannot be said for a man. In fact, you generally won't see female genitals in an American film in anything short of porn or explicit erotica. This alleged double-standard is less a double standard than an admittedly depressing ability to come to terms culturally with the insides of women's bodies.'''
#success, final_query, query_count, final_prob = GS_WNR(classifier, text, 0)
#print(success, final_query, query_count, final_prob)



[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
